In [0]:
import pandas as pd
import numpy as np
bonds = pd.read_csv("https://raw.githubusercontent.com/alicexja/BankingAnalyticsData/master/bonds.csv")
mortgage = pd.read_csv("https://raw.githubusercontent.com/alicexja/BankingAnalyticsData/master/mortgage.csv")
retail = pd.read_csv("https://raw.githubusercontent.com/alicexja/BankingAnalyticsData/master/retailloan.csv")
bond_yield = [0.0147,0.015,0.0137,0.0132]

In [0]:
bonds.PD = bonds.PD.apply(lambda x: x.strip('%'))
bonds.PD = bonds.PD.astype(float)/100

In [0]:
# convert pd,lgd and interest rate from str to numeric value
mortgage.PD = mortgage.PD.apply(lambda x: x.strip('%'))
mortgage.PD = mortgage.PD.astype(float)/100
mortgage.LGD = mortgage.LGD.apply(lambda x: x.strip('%'))
mortgage.LGD = mortgage.LGD.astype(float)/100
mortgage.Rate = mortgage.Rate.apply(lambda x: x.strip('%'))
mortgage.Rate = mortgage.Rate.astype(float)/100

In [0]:
retail.PD = retail.PD.apply(lambda x: x.strip('%'))
retail.PD = retail.PD.astype(float)/100
retail.YearlyInterestRate = retail.YearlyInterestRate.apply(lambda x: x.strip('%'))
retail.YearlyInterestRate = retail.YearlyInterestRate.astype(float)/100

In [14]:
bonds.head()

,id,PurchasePrice,CurrentCoupon,CouponsRemaining,Time2Maturity,BondsHeld,YearlyRate,Principal,PD
0,1,92.489,7,73,29.310559,26000,1.00,157,0.0215
1,2,101.676,2,13,2.272379,5000,0.98,101,0.1835
2,3,107.328,6,76,5.663777,100000,2.81,105,0.0065
3,4,127.260,6,74,25.764823,21000,0.87,179,0.6720
4,5,95.201,5,60,9.176237,5000001,0.77,97,0.3513


In [0]:
# bond pricing. This function calculates the price of bonds using market yield
def bpv(pr,c,n,bh,r=bond_yield):
  # pr:principal
  # c: coupon rate
  # n: time to maturity
  # bh: bonds held
  coupon = pr*c/100
  if round(n)<1:
    price = (pr/(1+r[0])**n)
    total_payment = price * bh/100
  else:
    f=n
    t=int(round(n))
    price = np.zeros(t+1)
    for i in range(t):
      if i+1<=3:
        price[i] = np.pv(rate=r[0], nper=i+1, pmt=0, fv=-c)
        price[t] = (pr/((1+r[0])**f))
      elif i+1<=5:
        price[i] = np.pv(rate=r[1], nper=i+1, pmt=0, fv=-c)
        price[t] = (pr/((1+r[1])**f))
      elif i+1<=10:
        price[i] = np.pv(rate=r[2], nper=i+1, pmt=0, fv=-c)
        price[t] = (pr/((1+r[2])**f))
      else:
        price[i] = np.pv(rate=r[3], nper=i+1, pmt=0, fv=-c)
        price[t] = (pr/((1+r[3])**f))
    total_payment = np.sum(price)*bh/100
  return total_payment


In [0]:
bonds['EAD']=bonds.apply(lambda x: bpv(x['Principal'],x['YearlyRate'],x['Time2Maturity'],x['BondsHeld']),axis=1)

In [20]:
bonds.head()

,id,PurchasePrice,CurrentCoupon,CouponsRemaining,Time2Maturity,BondsHeld,YearlyRate,Principal,PD,EAD
0,1,92.489,7,73,29.310559,26000,1.00,157,0.0215,3.401378e+04
1,2,101.676,2,13,2.272379,5000,0.98,101,0.1835,4.981165e+03
2,3,107.328,6,76,5.663777,100000,2.81,105,0.0065,1.132457e+05
3,4,127.260,6,74,25.764823,21000,0.87,179,0.6720,3.080370e+04
4,5,95.201,5,60,9.176237,5000001,0.77,97,0.3513,4.603979e+06


In [23]:
retail.head()

,ID,OriginalAmount,Term,YearlyInterestRate,CurrentInstallment,PD
0,1,8305,23,0.19,6,0.8623
1,2,3380,22,0.26,10,0.0036
2,3,6955,35,0.29,17,0.1223
3,4,13443,23,0.10,14,0.8399
4,5,4037,22,0.15,9,0.0103


In [0]:
# retail loan pricing
def retail_pricing(a,n,r,ci):
  # a:original amount
  # n:term
  # r:yearly interest rate 
  # ci:current installment

  # transform yearly rate to monthly rate
  i = ((1+r)**(1/12))-1
  pmt = np.pmt(i,n,-a)
  payment_series = np.repeat(pmt, n-ci+1)
  payment_series[0] -= pmt
  # calculate the price at time n-ci
  pvt = np.npv(i, payment_series)
  return pvt

In [26]:
retail['EAD']=retail.apply(lambda x:retail_pricing(x['OriginalAmount'],x['Term'],x['YearlyInterestRate'],x['CurrentInstallment']),axis=1)
retail.head()

,ID,OriginalAmount,Term,YearlyInterestRate,CurrentInstallment,PD,EAD
0,1,8305,23,0.19,6,0.8623,6397.880924
1,2,3380,22,0.26,10,0.0036,2019.382782
2,3,6955,35,0.29,17,0.1223,4212.467514
3,4,13443,23,0.10,14,0.8399,5554.504992
4,5,4037,22,0.15,9,0.0103,2509.391765


In [27]:
mortgage.head()

,ID,OriginalAmount,TermYears,Collateral_valuation,CurrentInstallment,Rate,PD,LGD
0,18101,2067863,35,678311,306,0.082,0.0047,0.498
1,7835,309683,22,259913,166,0.117,0.1761,0.279
2,26416,2998421,21,2218432,133,0.092,0.0144,0.571
3,7304,483754,23,246967,152,0.032,0.1682,0.406
4,14807,1808956,37,499590,93,0.114,0.0067,0.458


In [30]:
def mort_pricing(a,n,co,ci,r):
  # a: original amount
  # n: term years
  # co: collateral valudation
  # ci: current installment
  # r: rate
  i = ((1+r)**(1/12))-1
  n = n*12
  pmt = np.pmt(i,n,-a)
  payment_series = np.repeat(pmt, n-ci+1)
  payment_series[0] -= pmt
  # calculate the price at time n-ci
  pvt = np.npv(i, payment_series)-co
  pvt = max(pvt,0)
  return pvt

mortgage['EAD'] = mortgage.apply(lambda x :mort_pricing(x['OriginalAmount'],x['TermYears'],x['Collateral_valuation'],x['CurrentInstallment'],x['Rate']),axis=1)
mortgage.head()

,ID,OriginalAmount,TermYears,Collateral_valuation,CurrentInstallment,Rate,PD,LGD,EAD
0,18101,2067863,35,678311,306,0.082,0.0047,0.498,4.852612e+05
1,7835,309683,22,259913,166,0.117,0.1761,0.279,0.000000e+00
2,26416,2998421,21,2218432,133,0.092,0.0144,0.571,0.000000e+00
3,7304,483754,23,246967,152,0.032,0.1682,0.406,1.378896e+04
4,14807,1808956,37,499590,93,0.114,0.0067,0.458,1.264946e+06


In [0]:
# apply floor on pd 0.03% and lgd 10%
def floor_pd_lgd(x,v):
  if x<v:
    return v
  else: return x
bonds['PD']=bonds.apply(lambda x:floor_pd_lgd(x['PD'],0.0003),axis=1)
retail['PD']=retail.apply(lambda x:floor_pd_lgd(x['PD'],0.0003),axis=1)
mortgage['PD']=mortgage.apply(lambda x:floor_pd_lgd(x['PD'],0.0003),axis=1)
mortgage['LGD']=mortgage.apply(lambda x:floor_pd_lgd(x['LGD'],0.01),axis=1)

In [0]:
# calculate expected loss
def expected_loss(ead,pd,lgd):
  return pd*lgd*ead

# for retail loan, loss given default is 1
retail['Expected_Loss']=retail.apply(lambda x:expected_loss(x['EAD'],x['PD'],lgd=1),axis=1)
mortgage['Expected_Loss']=mortgage.apply(lambda x:expected_loss(x['EAD'],x['PD'],x['LGD']),axis=1)
# for bonds, assume all of them are senior bonds,lgd=0.45
bonds['Expected_Loss']=bonds.apply(lambda x:expected_loss(x['EAD'],x['PD'],lgd=0.45),axis=1)


In [41]:
print(np.sum(retail.Expected_Loss))
print(np.sum(mortgage.Expected_Loss))
print(np.sum(bonds.Expected_Loss))

25602584.262574226
504665684.53099513
5814221106.928184


## Basel III Capital Requirements

Recalling the last lecture, the equation for the capital requirement of any operation is:

$$
K = LGD \cdot \left\{ N\left( \sqrt{\frac{1}{1-R}} \cdot N^{-1}(PD) + \sqrt{\frac{R}{1-R}} \cdot N^{-1}(0.999) \right) - PD \right\} \left( \frac{1 + (M + 2.5)b}{1 - 1.5b}\right)
$$

The values of $b$ and $M$ will be variable for bonds, but for retail and mortgages the maturity is fixed at 1, and the b term dissapears. The correlations are given by the regulation:

- Mortgages: $R = 0.15$
- Revolving: $R = 0.04$
- Other retail: $R = 0.03 \left( \frac{1 - e^{-35PD}}{1 - e^{-35}} \right) + 0.16 \left( 1 - \frac{1 - e^{-35PD}}{1 - e^{-35}} \right)$
- Corporate and sovereign exposures $ R = 0.12 \left( \frac{1 - e^{-50PD}}{1 - e^{-50}} \right) + 0.24 \left( 1 - \frac{1 - e^{-50PD}}{1 - e^{-50}} \right)$

With this we can calculate the capital requirements and the Risk Weighted Assets (RWA) for this portfolio. Let's start implementing the capital requirement function. Note that we require the cumulative normal distribution  and its inverse functions. For this we will use numpy's sister package [```scipy```](https://scipy.org/) which includes all (traditional) statistical models and quantities for classic stats (not analytics!) in its subpackage [```stats```](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html).

Within the package stats, we find the statistical distribution we need: [```norm```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm), the standard normal. Within it, we can call the cumulative function (```norm.cdf```) and the inverse function, ```norm.ppf``` which stands for *[percent point function](https://stackoverflow.com/questions/20626994/how-to-calculate-the-inverse-of-the-normal-cumulative-distribution-function-in-p)*.

In [0]:
# calculate capital requirement
import numpy as np
from scipy.stats import norm
def capital_requirement(PD, LGD, t,M=0):
  if t == "retail":
    R =  0.03 * ( (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) ) 
    R += 0.16 * (1 - ( (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) ) )
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
                 np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= LGD    
  elif t == "mortgage":
    R =  0.15 
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
                 np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= LGD 
  else:
    R =  0.12 * ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) ) 
    R += 0.24 * (1 - ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) ) )
    b = (0.11852 - 0.05478*np.log(PD))**2
    bm = (1+(M-2.5)*b)/(1-(1.5*b))
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= LGD * bm    
  return K

retail['Capital_Req'] = retail.apply(lambda x:capital_requirement(x['PD'],1,t="retail"),axis=1)
mortgage['Capital_Req'] = mortgage.apply(lambda x:capital_requirement(x['PD'],x['LGD'],t="mortgage"),axis=1)
bonds['Capital_Req'] = bonds.apply(lambda x:capital_requirement(x['PD'],0.45,t="bonds",M=x['Time2Maturity']),axis=1)

In [40]:
# calculate risk weighted assets
retail_rwa=12.5*np.dot(retail['Capital_Req'],retail['EAD'])
mortgage_rwa=12.5*np.dot(mortgage['Capital_Req'],mortgage['EAD'])
bonds_rwa=12.5*np.dot(bonds['Capital_Req'],bonds['EAD'])

capital_adequacy = (retail_rwa+mortgage_rwa+bonds_rwa)*0.115
capital_adequacy

12483008306.124186